# Merge all WNBA data

In [55]:
import pandas as pd

Get data from all CSVs + use the excel with the matching between team names and abbreviations in order to create new column, telling us what the opposing team is

In [56]:
general_game_data = pd.read_csv("/Users/patriciafonseca/Desktop/Tese/Data/WNBA/Final files/WNBA_2016_2022_shots.csv", index_col = 0)
rankings_WNBA = pd.read_csv("rankings_WNBA.csv", index_col = 0)
WNBA_2017_2023_DefenseRTG = pd.read_csv("WNBA_DefenseRTG.csv")
WNBA_Players_Characteristics = pd.read_csv("Updated_WNBA_Players_Characteristics2.csv")
team_names_df = pd.read_excel("team_names_wnba.xlsx").drop('Unnamed: 0', axis=1)

### Checking if team names and abbreviations are correct:

In [57]:
len(general_game_data["HOME_TEAM"].unique())

13

In [58]:
len(general_game_data["TEAM_NAME"].unique())

12

In [59]:
general_game_data[general_game_data["TEAM_NAME"] == "Las Vegas Aces"]

,WNBA GameID,TEAM_NAME,GAME_DATE,SEASON_1,SEASON_2,AWAY_TEAM,HOME_TEAM
4,1021600005,Las Vegas Aces,05-15-2016,2016,2015-16,ATL,SAN
10,1021600011,Las Vegas Aces,05-19-2016,2016,2015-16,CON,SAN
25,1021600026,Las Vegas Aces,05-28-2016,2016,2015-16,CHI,SAN
41,1021600042,Las Vegas Aces,06-05-2016,2016,2015-16,LAS,SAN
53,1021600054,Las Vegas Aces,06-12-2016,2016,2015-16,NYL,SAN
...,...,...,...,...,...,...,...
1179,1022200160,Las Vegas Aces,07-22-2022,2022,2021-22,IND,LVA
1184,1022200165,Las Vegas Aces,07-24-2022,2022,2021-22,LAS,LVA
1220,1022200201,Las Vegas Aces,08-10-2022,2022,2021-22,ATL,LVA
1224,1022200205,Las Vegas Aces,08-12-2022,2022,2021-22,CHI,LVA


Sometimes, the abbreviation for the Las Vegas Aces team appears as "LVA", and sometimes it appears as "SAN". Let's correct that:

In [60]:
general_game_data['HOME_TEAM'] = general_game_data['HOME_TEAM'].replace('SAN', 'LVA')

### Remove data from 2016 (we are only doing from 2017 forward)

In [61]:
WNBA_2017_2023_Shots = general_game_data[general_game_data['SEASON_1'] != 2016]

# If you want to reset the index after dropping rows
WNBA_2017_2023_Shots.reset_index(drop=True, inplace=True)

### Merge data from all years for play-by-play and spatial data

In [62]:
# Initialize an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Iterate through the years from 2017 to 2022
for year in range(2017, 2023):
    # Construct file names for PT1 and PT2 CSV files for the current year
    pt1_filename = f'WNBAShots_{year}_PT1_TESTED.csv'
    pt2_filename = f'WNBAShots_{year}_PT2_TESTED.csv'
    
    pt1_df = pd.read_csv(pt1_filename)
    pt2_df = pd.read_csv(pt2_filename)

    # Filter pt1_df based on "GAME_ID" values present in pt2_df
    pt1_df = pt1_df[pt1_df['GAME_ID'].isin(pt2_df['GAME_ID'])]

    # Create new column with time to ensure match between pt1 and pt2
    # Convert QUARTER, MINS_LEFT, and SECS_LEFT columns to strings
    pt1_df["QUARTER"] = pt1_df["QUARTER"].astype(str)
    pt1_df["MINS_LEFT"] = pt1_df["MINS_LEFT"].astype(str)

    # Convert SECS_LEFT to strings with leading zeros if less than 10
    pt1_df["SECS_LEFT"] = pt1_df["SECS_LEFT"].astype(int).apply(lambda x: str(x).zfill(2))

    # Create the "time" column by concatenating the strings with the specified format
    pt1_df["time"] = pt1_df["QUARTER"] + "-" + pt1_df["MINS_LEFT"] + ":" + pt1_df["SECS_LEFT"]
    
    # Ensure that time column has same format: in pt2_df in 2017 there is one value of 2-10:0, and the same play on pt1_df has 2-10:00 (game_id = 1021700165)
    # Replace ':0' with ':00' in the time column
    pt2_df["time"] = pt2_df["time"].str.replace(r':0$', ':00', regex=True)

    # Merge DataFrames on 'GAME_ID'
    merged_df = pd.merge(pt1_df, pt2_df,
                         how = "left",
                          right_on=['GAME_ID', "time"],
                          left_on=["GAME_ID", "time"])
    
    # Append the merged data to the overall merged_data DataFrame
    merged_data = pd.concat([merged_data, merged_df], ignore_index=True)


In [63]:
columns_to_drop = ['Unnamed: 0', 'player', 'EVENT_TYPE_y', 'SHOT_TYPE_y',
                   'PLAY_DESCRIPTION', 'Unnamed: 0_x', 'Unnamed: 0_y', "time"]

merged_data.drop(columns=columns_to_drop, inplace=True)

merged_data.rename(columns = {"EVENT_TYPE_x": "EVENT_TYPE", "SHOT_TYPE_x": "SHOT_TYPE", "DISTANCE": "SHOT_DISTANCE", "ZONE_NAME": "BASIC_ZONE"}, inplace = True)

Add column that is missing:

In [64]:
merged_data["SHOT_MADE"] = merged_data["EVENT_TYPE"].apply(lambda x: "TRUE" if x == "Made Shot" else "FALSE")

Adjust strings on SHOT_DISTANCE to match the NBA dataset format, and convert to integer:

In [65]:
merged_data['SHOT_DISTANCE'] = merged_data['SHOT_DISTANCE'].apply(lambda x: x[:-1] if isinstance(x, str) and x[:-1].isdigit() else x)
merged_data['SHOT_DISTANCE'] = merged_data['SHOT_DISTANCE'].astype(float).astype('Int64')

### Analyse problem related to spatial data having more rows than play-by-play data

In [66]:
# Assuming pt1_df is your DataFrame
game_subset = pt1_df[pt1_df["GAME_ID"] == 1022200166]

# Find duplicates in the subset based on all columns
game_subset[game_subset.duplicated()]

,GAME_ID,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,EVENT_TYPE,SHOT_TYPE,ZONE_NAME,LOC_X,LOC_Y,QUARTER,MINS_LEFT,SECS_LEFT,time


In [67]:
game_subset["time"].value_counts()

time
3-6:18    2
1-9:37    1
3-3:29    1
3-6:22    1
3-6:02    1
         ..
1-1:11    1
1-2:05    1
1-4:03    1
1-6:01    1
4-0:05    1
Name: count, Length: 137, dtype: int64

This game has two shots on the same time made by same player, which is impossible. That is why the spatial data (pt1) has more rows that the play-by-play data (pt2).

In [68]:
game_subset2 = pt2_df[pt2_df["GAME_ID"] == 1022200166]
game_subset2["time"].value_counts()

time
1-9:37    1
3-3:08    1
3-4:23    1
3-4:11    1
3-4:06    1
         ..
2-9:49    1
1-0:00    1
1-0:05    1
1-0:12    1
4-0:05    1
Name: count, Length: 137, dtype: int64

We can see two shots taken at the same time by the same player:

In [69]:
game_subset[game_subset["time"] == "3-6:18"]

,GAME_ID,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,EVENT_TYPE,SHOT_TYPE,ZONE_NAME,LOC_X,LOC_Y,QUARTER,MINS_LEFT,SECS_LEFT,time
14163,1022200166,1611661325,Indiana Fever,1631031,Queen Egbo,Missed Shot,2PT Field Goal,Restricted Area,0,-5,3,6,18,3-6:18
14164,1022200166,1611661325,Indiana Fever,1631031,Queen Egbo,Missed Shot,2PT Field Goal,Restricted Area,-5,8,3,6,18,3-6:18


In [70]:
game_subset2[game_subset2["time"] == "3-6:18"]

,Unnamed: 0,GAME_ID,time,player,EVENT_TYPE,SHOT_TYPE,DISTANCE,ACTION_TYPE,PLAY_DESCRIPTION
14156,14156,1022200166,3-6:18,Egbo,Missed Shot,2PT Field Goal,1',Putback Layup,MISS Egbo 1' Putback Layup


### Merge shot data and general game data

In [71]:
merged_data["GAME_ID"] = merged_data["GAME_ID"].astype(int)
general_game_data["WNBA GameID"] = general_game_data["WNBA GameID"].astype(int)

WNBA_2017_2023_Shots = pd.merge(merged_data, general_game_data,
                         how = "left",
                         left_on=["GAME_ID"],
                          right_on=['WNBA GameID'])

In [72]:
columns_to_drop = ['WNBA GameID', 'TEAM_NAME_y']

WNBA_2017_2023_Shots.drop(columns=columns_to_drop, inplace=True)

WNBA_2017_2023_Shots.rename(columns = {"TEAM_NAME_x": "TEAM_NAME"}, inplace = True)

In [73]:
# Reorder columns
WNBA_2017_2023_Shots = WNBA_2017_2023_Shots[['SEASON_1', 'SEASON_2', 'TEAM_ID', 'TEAM_NAME', 'PLAYER_ID',
                                             'PLAYER_NAME', 'GAME_DATE', 'GAME_ID', 'HOME_TEAM', 'AWAY_TEAM',
                                             'EVENT_TYPE', 'SHOT_MADE', 'ACTION_TYPE', 'SHOT_TYPE', 'BASIC_ZONE',
                                             'LOC_X', 'LOC_Y', 'SHOT_DISTANCE', 'QUARTER', 'MINS_LEFT', 'SECS_LEFT']]

Count null values per column:


In [74]:
WNBA_2017_2023_Shots.isnull().sum()

SEASON_1             0
SEASON_2             0
TEAM_ID              0
TEAM_NAME            0
PLAYER_ID            0
PLAYER_NAME          0
GAME_DATE            0
GAME_ID              0
HOME_TEAM            0
AWAY_TEAM            0
EVENT_TYPE           0
SHOT_MADE            0
ACTION_TYPE          0
SHOT_TYPE            0
BASIC_ZONE           0
LOC_X                0
LOC_Y                0
SHOT_DISTANCE    10173
QUARTER              0
MINS_LEFT            0
SECS_LEFT            0
dtype: int64

### Adding defense rating data to original data:

We should add the defense rating for the opposing team, and not the team making the shot. 

Firstly, add column to NBA_2004_2023_Shots that says the team name abbreviation for the team being considered in each row. This allows for a easier identification of which the opposing team is, since the columns "HOME_TEAM" and "AWAY_TEAM" are expressed in team name abbreviations, and not the full team name.

In [75]:
team_names_df

,Team,Acronym
0,Atlanta Dream,ATL
1,Connecticut Sun,CON
2,Chicago Sky,CHI
3,Dallas Wings,DAL
4,Las Vegas Aces,LVA
5,Indiana Fever,IND
6,New York Liberty,NYL
7,Washington Mystics,WAS
8,Minnesota Lynx,MIN
9,Los Angeles Sparks,LAS


In [76]:
WNBA_2017_2023_Shots = WNBA_2017_2023_Shots.merge(team_names_df, left_on = "TEAM_NAME", right_on = 'Team', how = "left")

In [77]:
WNBA_2017_2023_Shots.drop("Team", axis = 1, inplace = True)
WNBA_2017_2023_Shots.rename(columns = {"Acronym" : "TEAM_ABB"}, inplace = True)

In [78]:
WNBA_2017_2023_Shots

,SEASON_1,SEASON_2,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,GAME_DATE,GAME_ID,HOME_TEAM,AWAY_TEAM,...,ACTION_TYPE,SHOT_TYPE,BASIC_ZONE,LOC_X,LOC_Y,SHOT_DISTANCE,QUARTER,MINS_LEFT,SECS_LEFT,TEAM_ABB
0,2017,2016-17,1611661319,Las Vegas Aces,100812,Erika de Souza,05-13-2017,1021700001,NYL,SAN,...,Jump Shot,2PT Field Goal,Mid-Range,117,144,19,1,8,51,LVA
1,2017,2016-17,1611661319,Las Vegas Aces,100812,Erika de Souza,05-13-2017,1021700001,NYL,SAN,...,Fadeaway Jumper,2PT Field Goal,In The Paint (Non-RA),-50,87,10,1,8,08,LVA
2,2017,2016-17,1611661319,Las Vegas Aces,204324,Dearica Hamby,05-13-2017,1021700001,NYL,SAN,...,Jump Shot,2PT Field Goal,Mid-Range,143,141,20,1,7,03,LVA
3,2017,2016-17,1611661319,Las Vegas Aces,202641,Sydney Colson,05-13-2017,1021700001,NYL,SAN,...,Jump Shot,2PT Field Goal,Mid-Range,-91,160,18,1,6,44,LVA
4,2017,2016-17,1611661319,Las Vegas Aces,204330,Isabelle Harrison,05-13-2017,1021700001,NYL,SAN,...,Jump Shot,2PT Field Goal,Mid-Range,45,156,16,1,6,04,LVA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141113,2022,2021-22,1611661320,Los Angeles Sparks,1628882,Lexie Brown,07-05-2022,1022200128,LAS,PHO,...,Jump Shot,3PT Field Goal,Above the Break 3,91,226,24,4,7,46,LAS
141114,2022,2021-22,1611661320,Los Angeles Sparks,203014,Nneka Ogwumike,07-05-2022,1022200128,LAS,PHO,...,Jump Shot,2PT Field Goal,Mid-Range,-77,203,22,4,7,03,LAS
141115,2022,2021-22,1611661320,Los Angeles Sparks,1629478,Katie Lou Samuelson,07-05-2022,1022200128,LAS,PHO,...,Putback Layup,2PT Field Goal,Restricted Area,0,-5,1,4,5,35,LAS
141116,2022,2021-22,1611661320,Los Angeles Sparks,203014,Nneka Ogwumike,07-05-2022,1022200128,LAS,PHO,...,Driving Layup,2PT Field Goal,Restricted Area,8,15,2,4,4,55,LAS


In [79]:
# Function to get the opposing team's name
def get_opposing_team(row):
        opposing_team_abbreviation = row['AWAY_TEAM'] if row['TEAM_ABB'] == row['HOME_TEAM'] else row['HOME_TEAM']
        return opposing_team_abbreviation

In [80]:
# Apply the function to create the new column
WNBA_2017_2023_Shots['OPPOSING_TEAM_ABB'] = WNBA_2017_2023_Shots.apply(get_opposing_team, axis=1)

Also match each team name to its abbreviation in the dataframe containing the defense rating for each team:

In [81]:
WNBA_2017_2023_DefenseRTG

,SEASON_1,TEAM,DEF RTG
0,2017.0,MINNESOTA LYNX,94.4
1,2017.0,LOS ANGELES SPARKS,96.1
2,2017.0,NEW YORK LIBERTY,97.9
3,2017.0,CONNECTICUT SUN,100.3
4,2017.0,PHOENIX MERCURY,102.6
...,...,...,...
67,2022.0,PHOENIX MERCURY,104.1
68,2022.0,ATLANTA DREAM,99.8
69,2022.0,MINNESOTA LYNX,104.4
70,2022.0,LOS ANGELES SPARKS,106.7


Change format of strings in column TEAM so that they are not all in uppercase:

In [82]:
WNBA_2017_2023_DefenseRTG['TEAM'] = WNBA_2017_2023_DefenseRTG['TEAM'].str.title()

In [83]:
WNBA_2017_2023_DefenseRTG

,SEASON_1,TEAM,DEF RTG
0,2017.0,Minnesota Lynx,94.4
1,2017.0,Los Angeles Sparks,96.1
2,2017.0,New York Liberty,97.9
3,2017.0,Connecticut Sun,100.3
4,2017.0,Phoenix Mercury,102.6
...,...,...,...
67,2022.0,Phoenix Mercury,104.1
68,2022.0,Atlanta Dream,99.8
69,2022.0,Minnesota Lynx,104.4
70,2022.0,Los Angeles Sparks,106.7


In [84]:
WNBA_2017_2023_DefenseRTG["TEAM"].value_counts()

TEAM
Minnesota Lynx        6
Los Angeles Sparks    6
New York Liberty      6
Connecticut Sun       6
Phoenix Mercury       6
Washington Mystics    6
Dallas Wings          6
Seattle Storm         6
Atlanta Dream         6
Chicago Sky           6
Indiana Fever         6
Las Vegas Aces        5
San Antonio Stars     1
Name: count, dtype: int64

After some research, we found that the Las Vegas Aces used to be called San Antonio Stars. In 2017, the team is sometimes referred as LVA, and other times referred as SAN. 

In [85]:
WNBA_2017_2023_DefenseRTG[WNBA_2017_2023_DefenseRTG["SEASON_1"] == 2017.0]

,SEASON_1,TEAM,DEF RTG
0,2017.0,Minnesota Lynx,94.4
1,2017.0,Los Angeles Sparks,96.1
2,2017.0,New York Liberty,97.9
3,2017.0,Connecticut Sun,100.3
4,2017.0,Phoenix Mercury,102.6
5,2017.0,Washington Mystics,103.5
6,2017.0,Dallas Wings,107.6
7,2017.0,Seattle Storm,105.0
8,2017.0,Atlanta Dream,101.5
9,2017.0,Chicago Sky,105.7


Above, there is the data for the team SAN (San Antonio Stars) in 2017. 

Replace "SAN" values in HOME_TEAM and AWAY_TEAM columns with "LVA": 

In [86]:
WNBA_2017_2023_Shots["HOME_TEAM"].replace("SAN", "LVA", inplace = True)
WNBA_2017_2023_Shots["AWAY_TEAM"].replace("SAN", "LVA", inplace = True)
WNBA_2017_2023_Shots["TEAM_ABB"].replace("SAN", "LVA", inplace = True)
WNBA_2017_2023_Shots["OPPOSING_TEAM_ABB"].replace("SAN", "LVA", inplace = True)

Now, we can merge the data correctly:

In [87]:
WNBA_2017_2023_DefenseRTG = WNBA_2017_2023_DefenseRTG.merge(team_names_df, left_on = "TEAM", right_on = 'Team', how = "left")

In [88]:
WNBA_2017_2023_DefenseRTG[WNBA_2017_2023_DefenseRTG["SEASON_1"] == 2017.0]

,SEASON_1,TEAM,DEF RTG,Team,Acronym
0,2017.0,Minnesota Lynx,94.4,Minnesota Lynx,MIN
1,2017.0,Los Angeles Sparks,96.1,Los Angeles Sparks,LAS
2,2017.0,New York Liberty,97.9,New York Liberty,NYL
3,2017.0,Connecticut Sun,100.3,Connecticut Sun,CON
4,2017.0,Phoenix Mercury,102.6,Phoenix Mercury,PHO
5,2017.0,Washington Mystics,103.5,Washington Mystics,WAS
6,2017.0,Dallas Wings,107.6,Dallas Wings,DAL
7,2017.0,Seattle Storm,105.0,Seattle Storm,SEA
8,2017.0,Atlanta Dream,101.5,Atlanta Dream,ATL
9,2017.0,Chicago Sky,105.7,Chicago Sky,CHI


Let's add the correct team abbreviation for the Las Vegas Aces (the webscrapping didn't accurately retrieve this data):

In [89]:
WNBA_2017_2023_DefenseRTG["Acronym"].replace("SAN", "LVA", inplace = True)

In [90]:
WNBA_2017_2023_DefenseRTG.drop(["Team", "TEAM"], axis=1, inplace=True)
WNBA_2017_2023_DefenseRTG.rename(columns = {"Acronym" : "TEAM_ABB_DEF_RTG"}, inplace = True)

In [91]:
WNBA_2017_2023_DefenseRTG[WNBA_2017_2023_DefenseRTG["SEASON_1"] == 2017.0]

,SEASON_1,DEF RTG,TEAM_ABB_DEF_RTG
0,2017.0,94.4,MIN
1,2017.0,96.1,LAS
2,2017.0,97.9,NYL
3,2017.0,100.3,CON
4,2017.0,102.6,PHO
5,2017.0,103.5,WAS
6,2017.0,107.6,DAL
7,2017.0,105.0,SEA
8,2017.0,101.5,ATL
9,2017.0,105.7,CHI


In [92]:
# Ensure columns have the same type to ensure a correct merge
WNBA_2017_2023_DefenseRTG["TEAM_ABB_DEF_RTG"] = WNBA_2017_2023_DefenseRTG["TEAM_ABB_DEF_RTG"].astype(str)
WNBA_2017_2023_DefenseRTG["SEASON_1"] = WNBA_2017_2023_DefenseRTG["SEASON_1"].astype(int).astype(str)

WNBA_2017_2023_Shots["OPPOSING_TEAM_ABB"] = WNBA_2017_2023_Shots["OPPOSING_TEAM_ABB"].astype(str)
WNBA_2017_2023_Shots["SEASON_1"] = WNBA_2017_2023_Shots["SEASON_1"].astype(str)

# Merge dataframes
WNBA_2017_2023_Shots = pd.merge(WNBA_2017_2023_Shots, WNBA_2017_2023_DefenseRTG, 
                     how='left', 
                     left_on=['SEASON_1', 'OPPOSING_TEAM_ABB'], 
                     right_on=['SEASON_1', 'TEAM_ABB_DEF_RTG'])

In [93]:
WNBA_2017_2023_Shots.drop(["TEAM_ABB_DEF_RTG"], axis=1, inplace=True)

In [94]:
WNBA_2017_2023_Shots.isnull().sum()

SEASON_1                 0
SEASON_2                 0
TEAM_ID                  0
TEAM_NAME                0
PLAYER_ID                0
PLAYER_NAME              0
GAME_DATE                0
GAME_ID                  0
HOME_TEAM                0
AWAY_TEAM                0
EVENT_TYPE               0
SHOT_MADE                0
ACTION_TYPE              0
SHOT_TYPE                0
BASIC_ZONE               0
LOC_X                    0
LOC_Y                    0
SHOT_DISTANCE        10173
QUARTER                  0
MINS_LEFT                0
SECS_LEFT                0
TEAM_ABB                 0
OPPOSING_TEAM_ABB        0
DEF RTG                  0
dtype: int64

### Adding ranking data to original data:

In [95]:
rankings_WNBA = rankings_WNBA.merge(team_names_df, left_on = "TEAM_NAME", right_on = 'Team', how = "left")

In [96]:
rankings_WNBA.rename(columns = {"Acronym" : "TEAM_ABB_RANK"}, inplace = True)

In [97]:
# Ensure same type of variables
rankings_WNBA["TEAM_ABB_RANK"] = rankings_WNBA["TEAM_ABB_RANK"].astype(str)
rankings_WNBA["SEASON_1"] = rankings_WNBA["SEASON_1"].astype(str)

WNBA_2017_2023_Shots["SEASON_1"] = WNBA_2017_2023_Shots["SEASON_1"].astype(str)

# Merge dataframes
WNBA_2017_2023_Shots = pd.merge(WNBA_2017_2023_Shots, rankings_WNBA, 
                     how='left', 
                     left_on=['SEASON_1', 'TEAM_ABB'], 
                     right_on=['SEASON_1', 'TEAM_ABB_RANK'])

In [98]:
WNBA_2017_2023_Shots.drop(["Team", "TEAM_ABB_RANK"], axis=1, inplace=True)

In [99]:
WNBA_2017_2023_Shots.isnull().sum()

SEASON_1                 0
SEASON_2                 0
TEAM_ID                  0
TEAM_NAME_x              0
PLAYER_ID                0
PLAYER_NAME              0
GAME_DATE                0
GAME_ID                  0
HOME_TEAM                0
AWAY_TEAM                0
EVENT_TYPE               0
SHOT_MADE                0
ACTION_TYPE              0
SHOT_TYPE                0
BASIC_ZONE               0
LOC_X                    0
LOC_Y                    0
SHOT_DISTANCE        10173
QUARTER                  0
MINS_LEFT                0
SECS_LEFT                0
TEAM_ABB                 0
OPPOSING_TEAM_ABB        0
DEF RTG                  0
TEAM_NAME_y              0
RANKING                  0
CONFERENCE               0
dtype: int64

### Adding player characteristics data to original data:

In [100]:
WNBA_Players_Characteristics

,PLAYER_ID,FIRST_SEASON_PLAYED,BIRTHDATE
0,100812,2002,9-Mar-82
1,204324,2015,6-Nov-93
2,202641,2011,6-Aug-89
3,204330,2016,27-Sep-93
4,202652,2011,12-Nov-88
...,...,...,...
313,1631072,2022,25-Sep-00
314,1630471,2022,15-Dec-98
315,1631062,2022,25-Nov-98
316,1630387,2022,12-Jul-01


In [101]:
# Ensure same type of variables
WNBA_Players_Characteristics["PLAYER_ID"] = WNBA_Players_Characteristics["PLAYER_ID"].astype(int)

WNBA_2017_2023_Shots["PLAYER_ID"] = WNBA_2017_2023_Shots["PLAYER_ID"].astype(int)

# Merge dataframes
WNBA_2017_2023_Shots = pd.merge(WNBA_2017_2023_Shots, WNBA_Players_Characteristics, 
                     how='left', 
                     left_on=['PLAYER_ID'], 
                     right_on=['PLAYER_ID'])

In [102]:
WNBA_2017_2023_Shots

,SEASON_1,SEASON_2,TEAM_ID,TEAM_NAME_x,PLAYER_ID,PLAYER_NAME,GAME_DATE,GAME_ID,HOME_TEAM,AWAY_TEAM,...,MINS_LEFT,SECS_LEFT,TEAM_ABB,OPPOSING_TEAM_ABB,DEF RTG,TEAM_NAME_y,RANKING,CONFERENCE,FIRST_SEASON_PLAYED,BIRTHDATE
0,2017,2016-17,1611661319,Las Vegas Aces,100812,Erika de Souza,05-13-2017,1021700001,NYL,LVA,...,8,51,LVA,NYL,97.9,Las Vegas Aces,6,Western Conference,2002,9-Mar-82
1,2017,2016-17,1611661319,Las Vegas Aces,100812,Erika de Souza,05-13-2017,1021700001,NYL,LVA,...,8,08,LVA,NYL,97.9,Las Vegas Aces,6,Western Conference,2002,9-Mar-82
2,2017,2016-17,1611661319,Las Vegas Aces,204324,Dearica Hamby,05-13-2017,1021700001,NYL,LVA,...,7,03,LVA,NYL,97.9,Las Vegas Aces,6,Western Conference,2015,6-Nov-93
3,2017,2016-17,1611661319,Las Vegas Aces,202641,Sydney Colson,05-13-2017,1021700001,NYL,LVA,...,6,44,LVA,NYL,97.9,Las Vegas Aces,6,Western Conference,2011,6-Aug-89
4,2017,2016-17,1611661319,Las Vegas Aces,204330,Isabelle Harrison,05-13-2017,1021700001,NYL,LVA,...,6,04,LVA,NYL,97.9,Las Vegas Aces,6,Western Conference,2016,27-Sep-93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141113,2022,2021-22,1611661320,Los Angeles Sparks,1628882,Lexie Brown,07-05-2022,1022200128,LAS,PHO,...,7,46,LAS,PHO,104.1,Los Angeles Sparks,6,Western Conference,2018,27-Oct-94
141114,2022,2021-22,1611661320,Los Angeles Sparks,203014,Nneka Ogwumike,07-05-2022,1022200128,LAS,PHO,...,7,03,LAS,PHO,104.1,Los Angeles Sparks,6,Western Conference,2012,2-Jul-90
141115,2022,2021-22,1611661320,Los Angeles Sparks,1629478,Katie Lou Samuelson,07-05-2022,1022200128,LAS,PHO,...,5,35,LAS,PHO,104.1,Los Angeles Sparks,6,Western Conference,2019,13-Jun-97
141116,2022,2021-22,1611661320,Los Angeles Sparks,203014,Nneka Ogwumike,07-05-2022,1022200128,LAS,PHO,...,4,55,LAS,PHO,104.1,Los Angeles Sparks,6,Western Conference,2012,2-Jul-90


In [103]:
# Drop columns we don't need
WNBA_2017_2023_Shots.drop(columns=['TEAM_NAME_y'], inplace=True)
WNBA_2017_2023_Shots.rename(columns = {"BIRTHDATE" : "PLAYER_BIRTHDATE", "TEAM_NAME_x" : "TEAM_NAME"}, inplace = True)

In [104]:
WNBA_2017_2023_Shots

,SEASON_1,SEASON_2,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,GAME_DATE,GAME_ID,HOME_TEAM,AWAY_TEAM,...,QUARTER,MINS_LEFT,SECS_LEFT,TEAM_ABB,OPPOSING_TEAM_ABB,DEF RTG,RANKING,CONFERENCE,FIRST_SEASON_PLAYED,PLAYER_BIRTHDATE
0,2017,2016-17,1611661319,Las Vegas Aces,100812,Erika de Souza,05-13-2017,1021700001,NYL,LVA,...,1,8,51,LVA,NYL,97.9,6,Western Conference,2002,9-Mar-82
1,2017,2016-17,1611661319,Las Vegas Aces,100812,Erika de Souza,05-13-2017,1021700001,NYL,LVA,...,1,8,08,LVA,NYL,97.9,6,Western Conference,2002,9-Mar-82
2,2017,2016-17,1611661319,Las Vegas Aces,204324,Dearica Hamby,05-13-2017,1021700001,NYL,LVA,...,1,7,03,LVA,NYL,97.9,6,Western Conference,2015,6-Nov-93
3,2017,2016-17,1611661319,Las Vegas Aces,202641,Sydney Colson,05-13-2017,1021700001,NYL,LVA,...,1,6,44,LVA,NYL,97.9,6,Western Conference,2011,6-Aug-89
4,2017,2016-17,1611661319,Las Vegas Aces,204330,Isabelle Harrison,05-13-2017,1021700001,NYL,LVA,...,1,6,04,LVA,NYL,97.9,6,Western Conference,2016,27-Sep-93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141113,2022,2021-22,1611661320,Los Angeles Sparks,1628882,Lexie Brown,07-05-2022,1022200128,LAS,PHO,...,4,7,46,LAS,PHO,104.1,6,Western Conference,2018,27-Oct-94
141114,2022,2021-22,1611661320,Los Angeles Sparks,203014,Nneka Ogwumike,07-05-2022,1022200128,LAS,PHO,...,4,7,03,LAS,PHO,104.1,6,Western Conference,2012,2-Jul-90
141115,2022,2021-22,1611661320,Los Angeles Sparks,1629478,Katie Lou Samuelson,07-05-2022,1022200128,LAS,PHO,...,4,5,35,LAS,PHO,104.1,6,Western Conference,2019,13-Jun-97
141116,2022,2021-22,1611661320,Los Angeles Sparks,203014,Nneka Ogwumike,07-05-2022,1022200128,LAS,PHO,...,4,4,55,LAS,PHO,104.1,6,Western Conference,2012,2-Jul-90


In [107]:
WNBA_2017_2023_Shots.isnull().sum()

SEASON_1                   0
SEASON_2                   0
TEAM_ID                    0
TEAM_NAME                  0
PLAYER_ID                  0
PLAYER_NAME                0
GAME_DATE                  0
GAME_ID                    0
HOME_TEAM                  0
AWAY_TEAM                  0
EVENT_TYPE                 0
SHOT_MADE                  0
ACTION_TYPE                0
SHOT_TYPE                  0
BASIC_ZONE                 0
LOC_X                      0
LOC_Y                      0
SHOT_DISTANCE          10173
QUARTER                    0
MINS_LEFT                  0
SECS_LEFT                  0
TEAM_ABB                   0
OPPOSING_TEAM_ABB          0
DEF RTG                    0
RANKING                    0
CONFERENCE                 0
FIRST_SEASON_PLAYED        0
PLAYER_BIRTHDATE           0
dtype: int64

### Create new CSV file

In [108]:
WNBA_2017_2023_Shots.to_csv("WNBA_2017_2023_Shots_Extra_Features.csv")